# Aufgabe 10: 3-SAT Problem

Befehl env/Scripts/activate zum Starten der Virtuellen Umgebung


In [1624]:
import numpy as np
rng = np.random.default_rng()

################    Parameter   ################

anzahlVariablen = 20
anzahlKlauseln = 20
maxLiteraleProKlausel = 3

maxGenerationen = 30
populationsGroesse = 100
anzahlSelektion = 20
anzahlMutation = 20
anzahlRekombination = 30 #muss eine Gerade Zahl sein
anzahlNeueIndividuen = 30

################################################

In [1625]:
def createRandomKNF():      #erstellt eine zufällige KNF
    
    Klauselmenge = []
    for i in range(anzahlKlauseln):
        numLiterale = rng.integers(low=1, high=(maxLiteraleProKlausel + 1))
        Klausel = []
        for k in range(numLiterale):
            literal = rng.integers(low = 1, high=(anzahlVariablen + 1))
            literal = -1*literal if rng.integers(low=0, high=2) == 1 else literal
            while(literal in Klausel or -1*literal in Klausel):
                literal = rng.integers(low = 1, high=(anzahlVariablen + 1))
                literal = -1*literal if rng.integers(low=0, high=2) == 1 else literal
            Klausel.append(literal)
        for k in range(maxLiteraleProKlausel - numLiterale):
            Klausel.append(0)
        Klauselmenge.append(Klausel)
    
    return Klauselmenge

In [1626]:
def printKNF(knf):      #gibt eine KNF in darstellender Weise aus
    
    print('KNF:')
    ausgabe = ''
    for i, klausel in enumerate(knf):
        ausgabe += "("
        for literal in klausel:
            if literal == 0:
                continue
            else:
                variable = abs(literal)
                literal_string = ('x' if literal > 0 else '¬x') + ''.join(chr(8320 + int(digit)) for digit in str(variable))
                ausgabe += literal_string + ' ∨ '

        ausgabe = ausgabe.rstrip(' ∨ ')
        ausgabe += ")" if i == len(knf) - 1 else ") ∧ "

    print(ausgabe, '\n')

In [1627]:
def InitPopulation():       #liefert eine zufällige Anfangspopulation
    
    initPopulation = rng.integers(size=(populationsGroesse, (anzahlVariablen + 1)), low=0, high=2)
    for individuum in initPopulation:
        individuum[0] = 0

    return initPopulation

In [1628]:
def sortPopulation(population, knf):        #sortiert alle Individuen einer Generation nach der  Anzahl erfüllter Klauseln der KNF

    bewertung = {}  #dictionary speichert die Anzahl erfüllter Klauseln aller Individuen
    for i, individuum in enumerate(population):
        erfüllteKlauseln = 0
        for klausel in knf:
            for variable in klausel:
                if variable > 0 and individuum[variable] == 1:  #Variable positiv & Belegung 1
                    erfüllteKlauseln = erfüllteKlauseln + 1
                    break
                elif variable < 0 and individuum[-1 * variable] == 0:   #Variable negativ & Belegung 0
                    erfüllteKlauseln = erfüllteKlauseln + 1
                    break
        bewertung[i] = erfüllteKlauseln

    sortierteIndividuen = dict(sorted(bewertung.items(), key=lambda item: item[1]))     #sortiert das dictionary

    return sortierteIndividuen

In [1629]:
def Selektion(sortedPopulation, population):    #selektiert die besten Individuen aus einer Generation
    
    selected = []
    np.array(selected)
    for i in range(anzahlSelektion):
        individuum = population[list(sortedPopulation.keys())[(populationsGroesse - 1)-i]]
        
        selected.extend(individuum)
        
    selected = np.reshape(selected, (anzahlSelektion, (anzahlVariablen + 1)))
    
    return selected

In [1630]:
def Mutation(sortedPopulation, population):     #verändert eine zufällige Stelle in manchen Individuen

    mutatedPopulation = []
    for i in range(anzahlMutation):
        individuum = population[list(sortedPopulation.keys())[(populationsGroesse - 1)-i] ]
        mutationSpot = rng.integers(low=1, high=(anzahlVariablen + 1))
        individuum[mutationSpot] = 1 if individuum[mutationSpot] == 0 else 0
        mutatedPopulation.append(individuum)

    return mutatedPopulation

In [1631]:
def Rekombination(sortedPopulation, population):    #erstellt neue Individuen durch Rekombiantion aus zwei der aktuell besten Individuen
    
    recombinedPopulation = []
    for i in range(int(anzahlRekombination / 2)):
        individuum1 = np.array_split(population[list(sortedPopulation.keys())[(populationsGroesse - 1)-i]], 2)
        individuum2 = np.array_split(population[list(sortedPopulation.keys())[rng.integers(low=1, high=21)]], 2)

        new1 = np.concatenate((individuum1[0], individuum2[1]))
        new2 = np.concatenate((individuum2[0], individuum1[1]))

        recombinedPopulation.append(new1)
        recombinedPopulation.append(new2)
        
    return recombinedPopulation

In [1632]:
def createNewRandom():      #erstellt neue zufällige Individuen
    
    newRandoms = rng.integers(size=(anzahlNeueIndividuen, (anzahlVariablen+1)), low=0, high=2)
    for individuum in newRandoms:
        individuum[0] = 0
        
    return newRandoms

In [1633]:
def Evolution(knf, iterations, startPopulation):    #Programmablauf

    population = startPopulation

    for i in range(iterations):
        sortedPopulation = sortPopulation(population, knf)      #sortiert die Population der i. Generation

        if(list(sortedPopulation.values())[populationsGroesse - 1] == anzahlKlauseln):      #Abbruchkriterium (wenn eine Belegung gefunden wurde, für welche alle Klauseln wahr sind)
            print(f'Erfüllende Belegung gefunden in der {i}. Generation')
            return population[list(sortedPopulation.keys())[(populationsGroesse - 1)]]

        selektion = Selektion(sortedPopulation, population)
        mutation = Mutation(sortedPopulation, population)
        rekombination = Rekombination(sortedPopulation, population)
        newRandoms = createNewRandom()

        newPopulation = np.concatenate((selektion, newRandoms,mutation,rekombination ),  axis=0)    #erstellt eine neue Population aus den 4 angewandten Methoden

        population = newPopulation

    print(f'Nach {maxGenerationen} Generationen wurde keine Erfüllende Belegung gefunden')      #Es wurde keine Belegung gefunden -> wahrscheinlich eine Kontradiktion
    
    return ['Kontradiktion']

In [1634]:
def print_marked_knf(knf, belegung):    #gibt die KNF aus, wobei alle Literale, welche unter der Belung w sind, grün geschrieben werden
    GREEN = '\033[92m'
    RESET = '\033[0m'

    ausgabe = ''
    for i, klausel in enumerate(knf):
        ausgabe += "("
        for literal in klausel:
            if literal == 0:
                continue
            else:
                variable = abs(literal)
                literal_string = ('x' if literal > 0 else '¬x') + ''.join(chr(8320 + int(digit)) for digit in str(variable))
                if (literal > 0 and belegung[variable]) or (literal < 0 and not belegung[variable]):
                    ausgabe += GREEN + literal_string + RESET + ' ∨ '
                else:
                    ausgabe += literal_string + ' ∨ '

        ausgabe = ausgabe.rstrip(' ∨ ')
        ausgabe += ")" if i == len(knf) - 1 else ") ∧ "

    print(ausgabe)


In [1635]:
def printBelegung(belegung):    #gibt eine Belegung aus
    for i, variable in enumerate(belegung):
        if i == 0: continue
        
        print(f'x' + ''.join(chr(8320 + int(digit)) for digit in str(i)) + ':', ('w' if variable == 1 else 'f'), end='      ')  #gibt einzelne Varibale aus, z.B. x₁ = w

        if i%4 == 0: print('\n')    #nach jeweils 4 Variablen ein Zeilenumbruch

    print('\n')

Tautologie:

In [1636]:
#Erstellen einer Tautologie
knf = []
for i in range(anzahlKlauseln):
    knf.append([1, -1, 0])      #entspricht der Klausel (x₁ ∨ ¬x₁ ∨ f) -> immer Wahr

startPopulation = InitPopulation()
belegung = Evolution(knf, maxGenerationen, startPopulation)

Erfüllende Belegung gefunden in der 0. Generation


Kontradiktion:

In [1637]:
knf = [[1, 0, 0]]       #entspricht der Klausel (x₁ ∨ f ∨ f)
for i in range(anzahlKlauseln - 1):                                      #(x₁ ∨ f ∨ f) ∧ (¬x₁ ∨ f ∨ f) immer falsch
    knf.append([-1, 0, 0])      #entspricht der Klausel (¬x₁ ∨ f ∨ f)

startPopulation = InitPopulation()
belegung = Evolution(knf, maxGenerationen, startPopulation)

Nach 30 Generationen wurde keine Erfüllende Belegung gefunden


Zufällige KNF:

In [1638]:
knf = createRandomKNF()
printKNF(knf)

startPopulation = InitPopulation()
belegung = Evolution(knf, maxGenerationen, startPopulation)

if belegung[0] != 'Kontradiktion':
    printBelegung(belegung)
    print_marked_knf(knf, belegung)

KNF:
(¬x₁₄ ∨ ¬x₈) ∧ (¬x₁₉ ∨ ¬x₁₇ ∨ ¬x₁₀) ∧ (x₁₈) ∧ (¬x₁₄ ∨ x₁₉) ∧ (x₂₀) ∧ (x₇ ∨ ¬x₂₀ ∨ x₈) ∧ (¬x₆) ∧ (¬x₁ ∨ x₁₄ ∨ ¬x₁₁) ∧ (x₁₅ ∨ x₁₉) ∧ (x₈) ∧ (¬x₁₄) ∧ (x₂) ∧ (x₁₂ ∨ x₃ ∨ ¬x₁₉) ∧ (x₁₄ ∨ ¬x₁₉ ∨ x₇) ∧ (x₁₁) ∧ (x₁₀) ∧ (¬x₉ ∨ ¬x₆) ∧ (¬x₂ ∨ x₅) ∧ (¬x₁₇) ∧ (x₁₆ ∨ ¬x₁₇) 

Erfüllende Belegung gefunden in der 3. Generation
x₁: f      x₂: w      x₃: f      x₄: f      

x₅: w      x₆: f      x₇: f      x₈: w      

x₉: w      x₁₀: w      x₁₁: w      x₁₂: f      

x₁₃: f      x₁₄: f      x₁₅: w      x₁₆: f      

x₁₇: f      x₁₈: w      x₁₉: f      x₂₀: w      



(¬x₁₄ ∨ ¬x₈) ∧ (¬x₁₉ ∨ ¬x₁₇ ∨ ¬x₁₀) ∧ (x₁₈) ∧ (¬x₁₄ ∨ x₁₉) ∧ (x₂₀) ∧ (x₇ ∨ ¬x₂₀ ∨ x₈) ∧ (¬x₆) ∧ (¬x₁ ∨ x₁₄ ∨ ¬x₁₁) ∧ (x₁₅ ∨ x₁₉) ∧ (x₈) ∧ (¬x₁₄) ∧ (x₂) ∧ (x₁₂ ∨ x₃ ∨ ¬x₁₉) ∧ (x₁₄ ∨ ¬x₁₉ ∨ x₇) ∧ (x₁₁) ∧ (x₁₀) ∧ (¬x₉ ∨ ¬x₆) ∧ (¬x₂ ∨ x₅) ∧ (¬x₁₇) ∧ (x₁₆ ∨ ¬x₁₇)
